In [1]:
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
from xhistogram.xarray import histogram
import os
import time
from matplotlib.colors import LogNorm
import dask

In [2]:
listOfDir = os.listdir('/Data2/SubX/')
listOfDir = listOfDir[0:-1] # drop 'CWB
listOfDir

['ESRL', 'NASA', 'EMC', 'RSMAS', 'ECCC', 'NRL', 'CESM', 'NCEP']

In [3]:
subfolder_to_try = ['FIMr1p1','GEOS_V2p1','GEFS','CCSM4','GEPS5','GEPS6','hindcast','NESM','46LCESM1', '30LCAM5','CFSv2']

In [4]:
for ild in listOfDir:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            print(os.listdir('/Data2/SubX/'+ild+'/'+isb)[0])


sic_sfc.zip
pr_sfc
JJA
tas_2m
forecast
forecast
subX_reforecast_ECCC_1995081000_m01.tar
documentation.txt
hindcast
hindcast
hindcast


# the list above will look for the subfolder in the folder. I have 11 possibilities, one for each model. In particular

In [5]:
for ild in listOfDir:
    for isb in subfolder_to_try:
        if isb in os.listdir('/Data2/SubX/'+ild+'/'):
            print(ild+'/'+isb)


ESRL/FIMr1p1
NASA/GEOS_V2p1
EMC/GEFS
RSMAS/CCSM4
ECCC/GEPS5
ECCC/GEPS6
ECCC/hindcast
NRL/NESM
CESM/46LCESM1
CESM/30LCAM5
NCEP/CFSv2


## each of this subfloders are organized differently (in fact the third ECCC doesn't have a name for the submodel, but directly the folder hindcast. I assume that is the GEM model)

## Unfortunately I cannot use a general loop to treat each model, but I have to write a separate script for each of them

## the name of the variables *should* be the same: cape, tas_2m, tdps_2m, wap_500, pr_sfc

# ECCC/GEPS5/hindcast

### These models have one big folder for all variables, so I have to reorganize the script

In [12]:
#!ls /Data2/SubX/ECCC/hindcast/

In [8]:
# ds = xr.open_dataset('/Data2/SubX/ECCC/GEPS6/hindcast/hfls_sfc_ECCC_01aug1998_00z_d01_d32_m01.nc')
# ds

In [10]:
variable_names = [ 'tas_2m', 'tdps_2m', 'wap_500', 'pr']

In [ ]:
from glob import glob
def pre_process_func(ds, strM):
#     print(strM)
#     print(ds)
    ds = ds.rename({'latitude':'lat', 'longitude':'lon'})
    ds = ds.sel(lat=slice(23,50), lon=slice(230,300))
    ds.coords['S'] = ds.time[0].values
    # FOR THIS MODEL ONLY I HAVE TO SUBSTRACT 24 HOURS
    ds.coords['S'] = ds.S - np.timedelta64(24,'h') 
#     print(ds.S)
    ds.coords['time']=np.array([  43200000000000,  129600000000000,  216000000000000,
        302400000000000,  388800000000000,  475200000000000,
        561600000000000,  648000000000000,  734400000000000,
        820800000000000,  907200000000000,  993600000000000,
       1080000000000000, 1166400000000000, 1252800000000000,
       1339200000000000, 1425600000000000, 1512000000000000,
       1598400000000000, 1684800000000000, 1771200000000000,
       1857600000000000, 1944000000000000, 2030400000000000,
       2116800000000000, 2203200000000000, 2289600000000000,
       2376000000000000, 2462400000000000, 2548800000000000,
       2635200000000000, 2721600000000000],
      dtype='timedelta64[ns]')

    ds.coords['M'] = np.int(strM)
#     ds.coords['time'] = ds.time-ds.S
    ds = ds.expand_dims(['S','M'])
    return ds
def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path, chunks={}) as ds:
#             print(path)
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds, path[-5:-3])
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
#             print(ds)
#             ds.load()
            return ds

    paths = sorted(glob(files))
#     print(paths)
    datasets = []
    for p in paths:
        try:
            datasets.append(process_one_path(p))
        except:
            print(p+' did not work')
            datasets=[]
        
#     datasets = [process_one_path(p) for p in paths]
    
    combined = xr.combine_by_coords(datasets)
    combined = combined.chunk({'S':4,'M':-1})
    return combined



var_l=[]
for ivar in variable_names:
    print(ivar)
    c_l = []
    for iy in np.arange(1999, 2015):
        print(iy)
        # here we suppose we only care about the combined mean of each file;
        # you might also use indexing operations like .sel to subset datasets
#             for im in np.arange()
        combined = read_netcdfs('/Data2/SubX/ECCC/hindcast/'+ivar+'*'+np.str(iy)+'*.nc', dim=['S','M'],
                                transform_func=pre_process_func)

        c_l.append(combined)
    all_values = xr.concat(c_l, dim='S') 
    var_l.append(all_values)
all_variables = xr.merge(var_l)

tas_2m
1999
2000
2001


In [ ]:
all_variables

In [ ]:
from glob import glob
def pre_process_func(ds, strM):
#     print(strM)
#     print(ds)
    ds = ds.rename({'latitude':'lat', 'longitude':'lon'})
    ds = ds.sel(lat=slice(23,50), lon=slice(230,300))
    ds.coords['S'] = ds.time[0].values
    # FOR THIS MODEL ONLY I HAVE TO SUBSTRACT 24 HOURS
    ds.coords['S'] = ds.S - np.timedelta64(24,'h') 
#     print(ds.S)
    ds.coords['time']=np.array([  43200000000000,  129600000000000,  216000000000000,
        302400000000000,  388800000000000,  475200000000000,
        561600000000000,  648000000000000,  734400000000000,
        820800000000000,  907200000000000,  993600000000000,
       1080000000000000, 1166400000000000, 1252800000000000,
       1339200000000000, 1425600000000000, 1512000000000000,
       1598400000000000, 1684800000000000, 1771200000000000,
       1857600000000000, 1944000000000000, 2030400000000000,
       2116800000000000, 2203200000000000, 2289600000000000,
       2376000000000000, 2462400000000000, 2548800000000000,
       2635200000000000, 2721600000000000],
      dtype='timedelta64[ns]')

    ds.coords['M'] = np.int(strM)
#     ds.coords['time'] = ds.time-ds.S
    ds = ds.expand_dims(['S','M'])
    return ds
def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path, chunks={}) as ds:
#             print(path)
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds, path[-5:-3])
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
#             print(ds)
#             ds.load()
            return ds

    paths = sorted(glob(files))
#     print(paths)
    datasets = []
    for p in paths:
        try:
            datasets.append(process_one_path(p))
        except:
            print(p+' did not work')
            datasets=[]
        
#     datasets = [process_one_path(p) for p in paths]
    
    combined = xr.combine_by_coords(datasets)
    combined = combined.chunk({'S':4,'M':-1})
    return combined



var_l=[]
for ivar in variable_names:
    print(ivar)
    c_l = []
    for iy in np.arange(1999, 2015):
        print(iy)
        # here we suppose we only care about the combined mean of each file;
        # you might also use indexing operations like .sel to subset datasets
#             for im in np.arange()
        combined = read_netcdfs('/Data2/SubX/ECCC/GEPS5/hindcast/'+ivar+'*'+np.str(iy)+'*.nc', dim=['S','M'],
                                transform_func=pre_process_func)

        c_l.append(combined)
    all_values = xr.concat(c_l, dim='S') 
    var_l.append(all_values)
all_variables = xr.merge(var_l)
all_variables

In [ ]:
from glob import glob
def pre_process_func(ds, strM):
#     print(strM)
#     print(ds)
    ds = ds.rename({'latitude':'lat', 'longitude':'lon'})
    ds = ds.sel(lat=slice(23,50), lon=slice(230,300))
    ds.coords['S'] = ds.time[0].values
    # FOR THIS MODEL ONLY I HAVE TO SUBSTRACT 24 HOURS
    ds.coords['S'] = ds.S - np.timedelta64(24,'h') 
#     print(ds.S)
    ds.coords['time']=np.array([  43200000000000,  129600000000000,  216000000000000,
        302400000000000,  388800000000000,  475200000000000,
        561600000000000,  648000000000000,  734400000000000,
        820800000000000,  907200000000000,  993600000000000,
       1080000000000000, 1166400000000000, 1252800000000000,
       1339200000000000, 1425600000000000, 1512000000000000,
       1598400000000000, 1684800000000000, 1771200000000000,
       1857600000000000, 1944000000000000, 2030400000000000,
       2116800000000000, 2203200000000000, 2289600000000000,
       2376000000000000, 2462400000000000, 2548800000000000,
       2635200000000000, 2721600000000000],
      dtype='timedelta64[ns]')

    ds.coords['M'] = np.int(strM)
#     ds.coords['time'] = ds.time-ds.S
    ds = ds.expand_dims(['S','M'])
    return ds
def read_netcdfs(files, dim, transform_func=None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path, chunks={}) as ds:
#             print(path)
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds, path[-5:-3])
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
#             print(ds)
#             ds.load()
            return ds

    paths = sorted(glob(files))
#     print(paths)
    datasets = []
    for p in paths:
        try:
            datasets.append(process_one_path(p))
        except:
            print(p+' did not work')
            datasets=[]
        
#     datasets = [process_one_path(p) for p in paths]
    
    combined = xr.combine_by_coords(datasets)
    combined = combined.chunk({'S':4,'M':-1})
    return combined



var_l=[]
for ivar in variable_names:
    print(ivar)
    c_l = []
    for iy in np.arange(1999, 2015):
        print(iy)
        # here we suppose we only care about the combined mean of each file;
        # you might also use indexing operations like .sel to subset datasets
#             for im in np.arange()
        combined = read_netcdfs('/Data2/SubX/ECCC/GEPS6/hindcast/'+ivar+'*'+np.str(iy)+'*.nc', dim=['S','M'],
                                transform_func=pre_process_func)

        c_l.append(combined)
    all_values = xr.concat(c_l, dim='S') 
    var_l.append(all_values)
all_variables = xr.merge(var_l)
all_variables